In [13]:
import flowermd
import hoomd
import numpy as np
from flowermd.library.forcefields import BeadSpring, KremerGrestBeadSpring
from flowermd.base import Pack, Simulation
from flowermd.library import LJChain
# importing tools

In [14]:
cpu = hoomd.device.CPU() # defining device for simulation

In [15]:
T = 1.5 # temperature for simulation
kg_chain = LJChain(lengths=10,num_mols=10) # creating a single polymer chain of length 10
ff = BeadSpring(r_cut=2.5,
        beads={"A": dict(epsilon=.1, sigma=1.0)},
        bonds={"A-A": dict(r0=1.0, k=5000.0)},
        angles={"A-A-A": dict(t0=np.pi, k=1.0)},
        dihedrals={"A-A-A-A": dict(phi0=0.0, k=0.1, d=-1, n=1)}) # forcefield with r0 = 1.0, as it is the initialized bond length. higher k 
        # will increase the stiffness of the chain
system = Pack(molecules = kg_chain, density=0.001) # creating a system with the polymer chain, low density is helpful for overlaps for multiple chains
sim = Simulation(
        initial_state=system.hoomd_snapshot,
        forcefield=ff.hoomd_forces,
        gsd_write_freq=int(1e5),
        device = cpu, gsd_file_name="single_chain.gsd") # passing in initial system snapshot, forcefield, writing frequency, device, and output file name
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=T) # initializing velocity of chain
sim.run_NVT(n_steps=1e6, kT=T, tau_kt=1) # running simulation 
sim.flush_writers() # logging data

/home/eridanirojas/miniconda3/envs/flowermd/lib/python3.12/site-packages/gmso/external/convert_hoomd.py:382: UserWarning: 90 bonds detected
  warnings.warn(f"{top.n_bonds} bonds detected")
/home/eridanirojas/miniconda3/envs/flowermd/lib/python3.12/site-packages/gmso/external/convert_hoomd.py:459: UserWarning: 80 angles detected
  warnings.warn(f"{top.n_angles} angles detected")
/home/eridanirojas/miniconda3/envs/flowermd/lib/python3.12/site-packages/gmso/external/convert_hoomd.py:505: UserWarning: 70 dihedrals detected
  warnings.warn(f"{top.n_dihedrals} dihedrals detected")


Initializing simulation state from a gsd.hoomd.Frame.
Step 50500 of 1000000; TPS: 63808.63; ETA: 0.2 minutes
Step 101000 of 1000000; TPS: 72052.64; ETA: 0.2 minutes
Step 151500 of 1000000; TPS: 74944.79; ETA: 0.2 minutes
Step 202000 of 1000000; TPS: 75802.75; ETA: 0.2 minutes
Step 252500 of 1000000; TPS: 76627.19; ETA: 0.2 minutes
Step 303000 of 1000000; TPS: 77172.72; ETA: 0.2 minutes
Step 353500 of 1000000; TPS: 77397.64; ETA: 0.1 minutes
Step 404000 of 1000000; TPS: 77450.17; ETA: 0.1 minutes
Step 454500 of 1000000; TPS: 77392.28; ETA: 0.1 minutes
Step 505000 of 1000000; TPS: 77332.53; ETA: 0.1 minutes
Step 555500 of 1000000; TPS: 77331.22; ETA: 0.1 minutes
Step 606000 of 1000000; TPS: 77416.32; ETA: 0.1 minutes
Step 656500 of 1000000; TPS: 77431.41; ETA: 0.1 minutes
Step 707000 of 1000000; TPS: 77406.27; ETA: 0.1 minutes
Step 757500 of 1000000; TPS: 77378.02; ETA: 0.1 minutes
Step 808000 of 1000000; TPS: 77369.28; ETA: 0.0 minutes
Step 858500 of 1000000; TPS: 77364.0; ETA: 0.0 minu

In [16]:
system.visualize() # visualizing chain

/home/eridanirojas/miniconda3/envs/flowermd/lib/python3.12/site-packages/mbuild/conversion.py:1994: UserWarning: No element attribute associated with '<A pos=([ 0.5655  5.2667 24.6489]), 1 bonds, id: 133989245342288>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [17]:
positions = system.hoomd_snapshot.particles.position
d = np.linalg.norm(positions[0] - positions[1])
print(f"Distance between particle 0 and 1: {d:.5f} nm")

Distance between particle 0 and 1: 1.00000 nm
